In [9]:
import nest_asyncio
from playwright.async_api import async_playwright
import pandas as pd

nest_asyncio.apply()

async def scrape_all_products_async():
    all_products = []

    pw = await async_playwright().start()
    browser = await pw.chromium.launch(headless=False)
    page = await browser.new_page()

    # All methods are async (use the "await" keyword)
    await page.goto("https://web-scraping.dev/products")

    while True:
        await page.wait_for_selector('.products')
        cards = await page.query_selector_all('.product')

        for card in cards:
            # Thumbnail image
            thumbnail = await card.query_selector('.thumbnail>img')

            # Description title and text
            title = await card.query_selector('.description>h3')
            text = await card.query_selector('.description>.short-description')

            # Price
            price = await card.query_selector('.price-wrap>.price')
            
            all_products.append({
                'thumbnail': await thumbnail.get_attribute('src') if thumbnail else '',
                'title': await title.inner_text() if title else '',
                'text': await text.inner_text() if text else '',
                'price': await price.inner_text() if price else ''
            })

        next_page = await page.query_selector('.paging>a:last-child') # Next page is the last button
        if(next_page and await next_page.get_attribute('href')): # Check if there is a link to the next page
            await next_page.click()
        else:
            break

    await browser.close()
    await pw.stop()

    return pd.DataFrame(all_products)

# Run the async main function
df = await scrape_all_products_async()  # Use await directly instead of asyncio.run()
df.to_csv("products.csv", index=False)
df.head()

,thumbnail,title,text,price
0,https://web-scraping.dev/assets/products/orang...,Box of Chocolate Candy,Indulge your sweet tooth with our Box of Choco...,24.99
1,https://web-scraping.dev/assets/products/darkr...,Dark Red Energy Potion,Unleash the power within with our 'Dark Red Po...,4.99
2,https://web-scraping.dev/assets/products/teal-...,Teal Energy Potion,Experience a surge of vitality with our 'Teal ...,4.99
3,https://web-scraping.dev/assets/products/red-p...,Red Energy Potion,"Elevate your game with our 'Red Potion', an ex...",4.99
4,https://web-scraping.dev/assets/products/blue-...,Blue Energy Potion,Ignite your gaming sessions with our 'Blue Ene...,4.99
